In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', 100) # Setting pandas to display a N number of columns
pd.set_option('display.max_rows', 10) # Setting pandas to display a N number rows
pd.set_option('display.width', 100) # Setting pandas dataframe display width to N
import matplotlib.pyplot as plt # plotting library
from src.utils import import_data, categorial_feature_overview, reduce_mem_usage, make_mi_scores, plot_mi_scores

import ppscore as pps # importing ppscore
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import plot_importance, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import pickle
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from src.utils import preprocess
%matplotlib inline

In [67]:
file = "usersessions/train.json"
df = pd.read_csv("train_processed.csv")
df = reduce_mem_usage(df)


Memory usage of dataframe is 12.06 MB
Memory usage after optimization is: 8.90 MB
Decreased by 26.2%


In [3]:
df[["dayofweek", "day", "month", "year"]] = df[
    ["dayofweek", "day", "month", "year"]
].astype(str)


In [4]:
useful_cols = df.drop(["time", "date_hour"], axis=1).columns
df = df[useful_cols]
cat_cols = df.select_dtypes(include=["object"]).columns
num_cols = df.select_dtypes(include=["number"]).columns



In [5]:
(df[cat_cols])


,browser,os,locale,gender,country,city,dayofweek,day,month,year,site_0,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_9,site_10,site_11,site_12
0,Chrome,Debian,vi-VN,m,France,Paris,1,6,12,2016,other,other,vk.com,other,other,other,other,other,other,other,other,other,other
1,Safari,MacOS,uk-UA,m,Australia,Sydney,4,16,12,2016,youtube.com,other,other,other,other,other,other,other,other,other,other,other,other
2,Firefox,Windows 10,pt-BR,f,Brazil,Rio de Janeiro,1,8,3,2016,other,other,other,other,other,other,other,other,other,other,other,other,other
3,Safari,MacOS,ur-PK,f,Malaysia,Kuala Lumpur,5,4,2,2017,youtube.com,toptal.com,other,other,other,other,other,other,other,other,other,other,other
4,Safari,MacOS,it-IT,m,Netherlands,Amsterdam,1,23,2,2016,youtube.com,slack.com,vk.com,other,other,other,other,other,other,other,other,other,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50995,Chrome,Windows 10,bg-BG,m,USA,New York,5,26,11,2016,other,other,other,other,other,other,other,toptal.com,other,other,other,other,other
50996,Chrome,Windows 8,zh-CN,m,France,Paris,2,27,7,2016,other,other,other,other,other,other,other,other,other,other,other,other,other
50997,Chrome,Debian,es-MX,m,Canada,Toronto,5,20,8,2016,other,other,other,other,other,other,other,other,other,other,other,other,other
50998,Chrome,Windows 8,uk-UA,f,Italy,Rome,0,24,7,2017,other,other,other,other,other,other,other,other,other,other,other,other,other


In [6]:
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

enc.fit(df[cat_cols])
pickle.dump(enc.fit(df[cat_cols]), open("encoder.pkl", 'wb'))

df[cat_cols] = enc.transform(df[cat_cols])

In [7]:
X = df.drop("is_joe", axis=1)
y = df["is_joe"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y.values, test_size=0.25, random_state=1, stratify=y
)


In [9]:
base_model = XGBClassifier()
base_model.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:34:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    classification_report,
)

predictions = base_model.predict(X_test)
print("ROC: " + str(roc_auc_score(y_test, predictions)))
print("F1: " + str(f1_score(y_test, predictions)))
print(classification_report(y_test, predictions))


ROC: 0.9756882296709107
F1: 0.9516129032258065
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12688
           1       0.95      0.95      0.95        62

    accuracy                           1.00     12750
   macro avg       0.98      0.98      0.98     12750
weighted avg       1.00      1.00      1.00     12750



In [11]:
from eli5 import (
    show_weights,
    show_prediction,
    explain_weights_xgboost,
    explain_prediction,
    format_as_dataframes,
)

show_weights(base_model)


Weight,Feature
0.2027,locale
0.0917,city
0.0878,browser
0.0786,hour_fixed
0.0755,gender
0.0748,site_0
0.0692,site_4
0.0614,site_1
0.0512,country
0.0362,site_2


In [12]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
k = 5
kfold = StratifiedKFold(k)
evalMetric = 'auc' 


def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "n_estimators": trial.suggest_int("n_estimators", 100,600,50),
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "learning_rate": trial.suggest_uniform("learning_rate", 1e-3, 1e-1),
        "subsample_range": trial.suggest_uniform("subsample_range", 1e-3, 99e-2),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 1e-3, 99e-2),
        "scale_pos_weight": trial.suggest_int('scale_pos_weight', 1, 100),
        "min_child_weight": trial.suggest_int('min_child_weight', 1, 20),

    }
    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
        param["grow_policy"] = trial.suggest_categorical(
            "grow_policy", ["depthwise", "lossguide"]
        )
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical(
            "sample_type", ["uniform", "weighted"]
        )
        param["normalize_type"] = trial.suggest_categorical(
            "normalize_type", ["tree", "forest"]
        )
        param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
        param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

    # Callback for pruning
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")

    
    # train_x, test_x, train_y, test_y = train_test_split(
    #     X, y, test_size=0.25, stratify=y
    # )
    # dtrain = xgb.DMatrix(train_x, label=train_y)
    # dtest = xgb.DMatrix(test_x, label=test_y)
    scores = []
    for train_index, test_index in kfold.split(X, y):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        bst = xgb.train(
            param, dtrain, evals=[(dtest, "validation")], callbacks = [pruning_callback]
        )
        preds = bst.predict(dtest)
        pred_labels = np.rint(preds)
        auc = roc_auc_score(y_test, pred_labels)
        scores.append(auc)

    return np.mean(scores)


In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=300)
print(study.best_trial)


[I 2022-01-31 16:34:55,563] A new study created in memory with name: no-name-70c64f58-51a1-4028-9fac-1aa6c6e1f7dc


[0]	validation-auc:0.95216
[1]	validation-auc:0.99068
[2]	validation-auc:0.98900
[3]	validation-auc:0.98521
[4]	validation-auc:0.99100
[5]	validation-auc:0.98973
[6]	validation-auc:0.99614
[7]	validation-auc:0.99555
[8]	validation-auc:0.99591
[9]	validation-auc:0.99658
[0]	validation-auc:0.95329
[1]	validation-auc:0.99238
[2]	validation-auc:0.99155
[3]	validation-auc:0.99201
[4]	validation-auc:0.99673
[5]	validation-auc:0.99654
[6]	validation-auc:0.99853
[7]	validation-auc:0.99832
[8]	validation-auc:0.99828
[9]	validation-auc:0.99839


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored becau

[0]	validation-auc:0.94335
[1]	validation-auc:0.98817
[2]	validation-auc:0.98804
[3]	validation-auc:0.99196
[4]	validation-auc:0.99601
[5]	validation-auc:0.99540
[6]	validation-auc:0.99704
[7]	validation-auc:0.99667
[8]	validation-auc:0.99726
[9]	validation-auc:0.99750


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored becau

...

[0]	validation-auc:0.99917
[1]	validation-auc:0.99856
[2]	validation-auc:0.99932


[I 2022-01-31 16:38:24,531] Trial 298 pruned. Trial was pruned at iteration 2.


[0]	validation-auc:0.99609


[I 2022-01-31 16:38:24,760] Trial 299 pruned. Trial was pruned at iteration 0.


FrozenTrial(number=241, values=[0.9896045562810846], datetime_start=datetime.datetime(2022, 1, 31, 16, 37, 49, 676165), datetime_complete=datetime.datetime(2022, 1, 31, 16, 37, 51, 123514), params={'n_estimators': 150, 'booster': 'dart', 'lambda': 0.007737405184614991, 'alpha': 0.3428088450353129, 'learning_rate': 0.02681205434630792, 'subsample_range': 0.055693273672147006, 'colsample_bytree': 0.6269097614090643, 'scale_pos_weight': 98, 'min_child_weight': 20, 'max_depth': 9, 'eta': 1.1185762241797551e-06, 'gamma': 0.023265978079085894, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 6.849371354624124e-05, 'skip_drop': 0.32756362421134955}, distributions={'n_estimators': IntUniformDistribution(high=600, low=100, step=50), 'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'learning_rate': UniformDistributi

In [14]:
params = study.best_params
params


{'n_estimators': 150,
 'booster': 'dart',
 'lambda': 0.007737405184614991,
 'alpha': 0.3428088450353129,
 'learning_rate': 0.02681205434630792,
 'subsample_range': 0.055693273672147006,
 'colsample_bytree': 0.6269097614090643,
 'scale_pos_weight': 98,
 'min_child_weight': 20,
 'max_depth': 9,
 'eta': 1.1185762241797551e-06,
 'gamma': 0.023265978079085894,
 'grow_policy': 'lossguide',
 'sample_type': 'weighted',
 'normalize_type': 'forest',
 'rate_drop': 6.849371354624124e-05,
 'skip_drop': 0.32756362421134955}

In [15]:
from xgboost import plot_importance


In [16]:
xgb_optuna = xgb.XGBClassifier(**params)
xgb_importance = xgb_optuna.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [17]:
show_weights(xgb_optuna)

Weight,Feature
0.2856,site_0
0.2778,locale
0.0642,site_1
0.0580,gender
0.0456,site_2
0.0448,hour_fixed
0.0306,browser
0.0260,os
0.0251,site_3
0.0187,site_6


In [18]:
kf = StratifiedKFold(5)
scores = []

xgb_optuna = xgb.XGBClassifier(use_label_encoder=False, **params)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_optuna.fit(X_train, y_train)
    preds = xgb_optuna.predict(X_test)
    pred_labels = np.rint(preds)
    auc = roc_auc_score(y_test, pred_labels)
    scores.append(auc)
    print(auc)
print("*" * 50)

print(np.mean(scores))


0.9584448300056896
0.9991625615763547
0.9689655172413792
0.9889655172413794
0.9791133004926107
**************************************************
0.9789303453114828


In [19]:
filename = "final_model.pkl"
# pickle.dump(xgb_optuna, open(filename, 'wb'))


In [20]:
kf = StratifiedKFold(5)
scores = []

xgb_xd = xgb.XGBClassifier(use_label_encoder=False)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_xd.fit(X_train, y_train)
    preds = xgb_xd.predict(X_test)
    pred_labels = np.rint(preds)
    auc = roc_auc_score(y_test, pred_labels)
    scores.append(auc)
    print(auc)
print("*" * 50)

print(np.mean(scores))


0.9183673469387755
0.97
0.8998029556650247
0.9598522167487683
0.9099014778325123
**************************************************
0.9315847994370163


In [36]:
xd = pd.read_csv("val.csv")
xd

,browser,os,locale,user_id,gender,location,sites,time,date
0,Firefox,Windows 8,wo-SN,166,f,Singapore/Singapore,"[{'site': 'microsoft.com', 'length': 52}, {'si...",03:40:00,2016-04-29
1,Safari,MacOS,pt-PT,51,m,Australia/Sydney,"[{'site': 'bing.com', 'length': 59}, {'site': ...",22:47:00,2016-08-31
2,Chrome,Debian,pt-BR,113,f,New Zealand/Auckland,"[{'site': 'baidu.com', 'length': 107}, {'site'...",07:06:00,2016-07-10
3,Firefox,Windows 8,wo-SN,160,f,USA/New York,"[{'site': 'vimeo.com', 'length': 100}, {'site'...",21:44:00,2016-01-15
4,Chrome,Windows 8,de-DE,23,m,Germany/Berlin,"[{'site': 'toptal.com', 'length': 60}, {'site'...",01:46:00,2016-02-28
...,...,...,...,...,...,...,...,...,...
8995,Firefox,Windows 10,xh-ZA,124,f,Spain/Madrid,"[{'site': 'jboss.org', 'length': 214}]",07:13:00,2017-09-12
8996,Firefox,Windows 8,uk-UA,118,f,Brazil/Rio de Janeiro,"[{'site': 'mangafox.me', 'length': 106}, {'sit...",15:00:00,2016-03-03
8997,Chrome,Windows 8,en-AU,37,m,New Zealand/Auckland,"[{'site': 'twitter.com', 'length': 109}, {'sit...",21:54:00,2016-11-28
8998,Firefox,Windows 8,ur-PK,36,m,UK/London,"[{'site': 'disqus.com', 'length': 80}, {'site'...",10:44:00,2017-04-22


In [37]:
from src.preprocessing import unwrap_sites

In [55]:
history = val["sites"].apply(pd.Series)
for i in history.columns:
    val["site_" + str(i)] = (
        history.fillna("").apply(lambda x: x.str["site"])[i].apply(clean_sites)
    )


In [45]:
fav_sites = [
    "blogadda.com",
    "jeuxonline.info",
    "lenta.ru",
    "mail.google.com",
    "slack.com",
    "toptal.com",
    "ucoz.net",
    "vk.com",
    "youtube.com",
]

In [46]:
def clean_sites(x):
    if x in fav_sites:
        return x
    elif x == "0":
        return "empty"
    else:
        return "other"


In [52]:
val = pd.read_csv("val.csv")

In [50]:
val = unwrap_sites(val)

In [79]:
from xgboost import DMatrix


In [87]:
enc = pickle.load(open("encoder.pkl", 'rb'))
model =  pickle.load(open("final_model.pkl", 'rb'))
val = pickle.load(open("val.pkl", 'rb'))

In [88]:
val = preprocess(val)

In [89]:
val[cat_cols] = enc.transform(val[cat_cols])

In [90]:
y_val = val.pop("is_joe")
x_val = val.drop(["date_hour","time"], axis=1)


In [93]:
x_val

,browser,os,locale,gender,country,city,dayofweek,day,month,year,hour,hour_fixed,sites_num,session_len,max_len_site,site_0,site_1,site_2,site_3,site_4,site_5,site_6,site_7,site_8,site_9,site_10,site_11,site_12
12628,1.0,5.0,21.0,0.0,13.0,16.0,4.0,21.0,6.0,0.0,3.666667,11.666667,7,747.0,247.0,2.0,2.0,9.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
37730,3.0,1.0,15.0,1.0,0.0,17.0,3.0,0.0,11.0,0.0,22.783333,9.783333,11,1433.0,364.0,2.0,2.0,4.0,4.0,4.0,4.0,9.0,4.0,4.0,2.0,3.0,2.0,1.0
39991,0.0,0.0,14.0,0.0,11.0,1.0,6.0,1.0,9.0,0.0,7.100000,20.100000,4,448.0,140.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
8525,1.0,5.0,21.0,0.0,16.0,10.0,4.0,6.0,0.0,0.0,21.733333,16.733333,8,1313.0,491.0,2.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,4.0,2.0,3.0,2.0,1.0
8279,0.0,5.0,1.0,1.0,5.0,2.0,6.0,20.0,4.0,0.0,1.766667,2.766667,6,781.0,174.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53785,1.0,3.0,22.0,0.0,14.0,7.0,1.0,3.0,11.0,1.0,7.216667,8.216667,1,214.0,214.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
57590,1.0,5.0,18.0,0.0,1.0,12.0,3.0,22.0,5.0,0.0,15.000000,12.000000,7,780.0,204.0,2.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
52646,0.0,5.0,2.0,1.0,11.0,1.0,1.0,21.0,2.0,0.0,21.900000,10.900000,11,1267.0,209.0,2.0,2.0,4.0,8.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0
15851,1.0,5.0,19.0,1.0,15.0,6.0,5.0,14.0,6.0,1.0,10.733333,10.733333,6,562.0,169.0,2.0,2.0,8.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,3.0,2.0,1.0


In [94]:
dval = x_val


In [99]:
preds = model.predict(dval)
preds.sum()


91

In [100]:
y_val.sum()

45

In [103]:
print(classification_report(y_val, preds))
print("ROC: " + str(roc_auc_score(y_val, preds)))
print("F1: " + str(f1_score(y_val, preds)))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      8955
           1       0.48      0.98      0.65        45

    accuracy                           0.99      9000
   macro avg       0.74      0.99      0.82      9000
weighted avg       1.00      0.99      1.00      9000

ROC: 0.9862646566164155
F1: 0.6470588235294118
